In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier


In [ ]:
df = pd. read_csv("C:/Users/kavya/OneDrive/Desktop/Spring2023_890/LoanFHLBData/original/LoanData_preprocessed.csv")

In [ ]:
target=["BoRace"]
X=df.loc[:,~df.columns.isin(target)]
y=df.loc[:,df.columns.isin(target)]
y = np.ravel(y)
feature_names=X.columns

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
scaler = MinMaxScaler(feature_range=(0, 1))


In [ ]:
selector = SelectKBest(chi2, k=10)
X_new = selector.fit_transform(X_scaled_df, y)
selected_features=selector.get_support(indices=True)
print([feature_names[i] for i in selected_features])

In [ ]:

class_labels = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}
# y[target] = y[target].map(class_labels)
# Map the year values to class labels
y[target] = y[target].applymap(class_labels.get)

X_train, X_test, y_train, y_test = train_test_split( X_scaled_df, y, test_size=0.30, random_state=0)

In [ ]:
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=13, learning_rate=0.1, max_depth=6, n_estimators=100, colsample_bytree=0.8)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [ ]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
dm=pd.DataFrame()
dm['Actual']=y_test
dm['Predicted']=y_pred

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
print("Accuracy:", accuracy)


In [ ]:
importances = xgb_model.feature_importances_
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})

# sort features by importance in descending order
feature_importances = feature_importances.sort_values(by='importance', ascending=False)
print(feature_importances)
# plot feature importances
plt.barh(feature_importances['feature'], feature_importances['importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importances')
plt.show()

In [ ]:
from scipy.stats import f_oneway
groups=[]
for col in feature_names:
        groups.append(df[col])
f_statistic, p_value = f_oneway(*groups)


In [ ]:
print(f_statistic, p_value )